# AquiMod2 Simulation Pipeline
This notebook runs AquiMod2 simulations, adjusts model configuration files, and plots GWL outputs from component results.

## 1. Setup and Imports

- Set working directory
- Import required helper functions
- Define paths to scenario and executable

In [65]:
import os
import importlib
import ipywidgets as widgets
from ipywidgets import interact
import pandas as pd

importlib.reload(utils.aquimod2_helpers)

from utils.aquimod2_helpers import (
    run_aquimod2,
    update_output_flags,
    update_input_option,
    load_component_gwl_output,
    plot_gwl_results,
    interactive_component_plot,
    get_input_value,
    summarize_input_file,
    interactive_file_plot
)

# Make sure we are in the correct root folder 
# Set working dir only if running from notebooks/
if os.path.basename(os.getcwd()) == "notebooks":
    os.chdir("..")
print("Current working directory:", os.getcwd())

scenario_path = os.path.abspath("scenarios/CHexample")
aquimod_exe_path = os.path.abspath("AquiMod2.exe")
input_path = os.path.join(scenario_path, "Input.txt")

print("Scenario path:", scenario_path)
print("AquiMod2 path:", aquimod_exe_path)
print("Input.txt path:", scenario_path)

Current working directory: D:\Github\Aquimod2
Scenario path: D:\Github\Aquimod2\scenarios\CHexample
AquiMod2 path: D:\Github\Aquimod2\AquiMod2.exe
Input.txt path: D:\Github\Aquimod2\scenarios\CHexample


## 2. Update Input.txt Settings

Use the `update_input_option()` function to dynamically change any section of the `Input.txt` file.

This allows full control over simulation settings such as:
- Simulation mode (`e`, `m`, `s`)
- Monte Carlo / SCE-UA calibration parameters
- Output file flags
- Objective function
- Spin-up period
...and more

This keeps workflow fast and reproducible — no manual edits required.

## 🔧 Input.txt Field Options (Quick Reference)

### Simulation mode
- `m` → Monte Carlo calibration
- `s` → SCE-UA calibration
- `e` → Evaluation using parameter samples (no calibration)

---

### Monte Carlo parameters
Format: `num_samples acceptance_ratio max_iterations g`
- Example: `100000 0.7 1000 g`

---

### SCE-UA parameters
Format: `max_iterations max_evaluations p m g`
- Use `-1 -1 -1 -1 g` for default behavior

---

### Evaluation parameters
- Typically: `1 g`  
- `1` refers to number of samples

---

### Objective function and parameters
- `1` = RMSE (Root Mean Squared Error)
- Future options: `2` (Bias), `3` (R²), etc. depending on implementation

---

### Spin-up period
- Integer (number of months)  
- Example: `6` = 6-month warm-up period

---

### Write model output files
- Format: `Y/N Y/N Y/N` for:
  - Soil moisture
  - Groundwater level
  - Component time series
- Example: `N Y Y` = only write GWL and component time series

---


In [58]:
# Define scenario input path
input_path = os.path.join(scenario_path, "Input.txt")

# Define the input parameters to update
input_edits = {
    "Simulation mode": "m",
    "Monte Carlo parameters": "100000 0.7 1000 g",
    "Write model output files": "Y Y Y",
    "Objective function and parameters": "1",
    "Spin-up period": "6"
}

# Apply each change to Input.txt
for key, val in input_edits.items():
    update_input_option(input_path, key, val)


Updated 'Simulation mode' to: m
Updated 'Monte Carlo parameters' to: 100000 0.7 1000 g
Updated 'Write model output files' to: Y Y Y
Updated 'Objective function and parameters' to: 1
Updated 'Spin-up period' to: 6


## 3. Run AquiMod2 Simulation

This cell launches the AquiMod2 model using the defined scenario.


In [59]:
run_aquimod2(aquimod_exe_path, scenario_path)

Running AquiMod2 in: D:\Github\Aquimod2\scenarios\CHexample
Simulation completed successfully.
Output: 

Running 100000 times...

Estimated time to finish = 9 seconds.     
Estimated time to finish = 1 seconds.     


Complete.

Extracting best model(s)...complete.

Writing output files...complete.

Model run time = 23 seconds.





## 4. Load Component Output and Plot GWL

This block loads the groundwater level (GWL) time series from the component output file (e.g., `Q3K3S1_TimeSeries1.out`) and plots the results.

The helper function `load_component_gwl_output()` reads the file and extracts the GWL column along with date formatting.

You can switch components by changing the filename (e.g., to `FAO_TimeSeries1.out` or `Weibull_TimeSeries1.out`).


In [69]:
interactive_component_plot(df)

interactive(children=(Dropdown(description='Plot:', options=('Q_3(m3/d)', 'Q_2(m3/d)', 'Q_1(m3/d)', 'GWL(m)'),…

In [66]:
interactive_file_plot(scenario_path)

interactive(children=(Dropdown(description='File:', options=('FAO_calib.out', 'FAO_TimeSeries1.out', 'fit_cali…

## ✅ Summary

- Ran AquiMod2 simulation with mode set in `Input.txt` (e.g., evaluation, calibration)
- Dynamically updated multiple `Input.txt` settings via helper functions
- Loaded component output file(s) such as `Q3K3S1_TimeSeries1.out`
- Visualized simulation results using a **dropdown selector** for flexible column plotting
- Built a reusable plotting block for future model outputs


In [67]:
# Get current simulation mode from Input.txt
sim_mode = get_input_value(input_path, "Simulation mode")

# Label for human-readable output
mode_label = {
    "e": "Evaluation",
    "m": "Monte Carlo Calibration",
    "s": "SCE-UA Calibration"
}.get(sim_mode.lower(), f"Unknown Mode: {sim_mode}")

print("🔧 Simulation mode detected:", mode_label)


🔧 Simulation mode detected: Monte Carlo Calibration


In [68]:
summarize_input_file(input_path)

📋 AquiMod2 Configuration Summary
----------------------------------------
Simulation mode: m
Monte Carlo parameters: 100000 0.7 1000 g
SCE-UA parameters: -1 -1 -1 -1 g
Evaluation parameters: 1 g
Objective function and parameters: 1
Spin-up period: 6
Write model output files: Y Y Y
